In [5]:
import numpy as np 
import pandas as pd 
import warnings
import re
import os
from typing import List
from pathlib import Path

warnings.filterwarnings(
    action='ignore',
    category=RuntimeWarning,
    module=r'pandas\.io\.formats\.format'
)
# file_list = []
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#        file_list.append(os.path.join(dirname, filename))


In [ ]:
PATH_ROOT = Path.cwd().parent
PATH_DATA = Path(f"{PATH_ROOT}\data")

In [ ]:
data_frames = {}

for file_path in PATH_DATA.glob('*.csv'):

    key_name = file_path.stem 
    
    df = pd.read_csv(file_path)
    
    data_frames[key_name] = df
  
    print(f"Geladen: {key_name} -> Shape: {df.shape}")


Geladen: consumer_price_index -> Shape: (4195, 10)
Geladen: manifesto -> Shape: (5285, 175)
Geladen: poverty_risk -> Shape: (8496, 11)
Geladen: real_gdp_per_capita -> Shape: (888, 10)
Geladen: sdg_08_10_page_linear -> Shape: (888, 10)
Geladen: unemployment_rate -> Shape: (1083, 12)
Geladen: wealth_distribution_gini -> Shape: (323, 10)


C:\Users\ssick\AppData\Local\Temp\ipykernel_70096\1701479870.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [21]:
PRICES_COLUMNS = ["geo", "TIME_PERIOD", "OBS_VALUE"]
df_prices = data_frames["consumer_price_index"][PRICES_COLUMNS]
df_prices = (
    df_prices
    .rename(columns={"OBS_VALUE": "consumer_prices"})
    .assign(year = lambda x: pd.to_numeric(x["TIME_PERIOD"], errors='coerce').astype("Int64"))
    .drop(columns=["TIME_PERIOD"])
    .set_index(["geo", "year"])
)
df_prices.head(10)


consumer_prices
geo     year                 
Albania 2016           101.51
        2017           104.76
        2018           106.59
        2019           108.39
        2020           110.74
        2021           113.26
        2022           120.69
        2023           127.13
        2024           130.38
Austria 2015           100.00

In [22]:
UNEMPLOYMENT_COLUMNS = ["geo", "TIME_PERIOD", "age", "sex", "OBS_VALUE"]
df_unemployment = data_frames["unemployment_rate"][UNEMPLOYMENT_COLUMNS]
sex_mapping = {
    "females": "female",
    "males": "male"}

df_unemployment = (
    df_unemployment
    .rename(columns={"OBS_VALUE": "unemployment_rate"})
   .assign(year = lambda x: pd.to_numeric(x["TIME_PERIOD"], errors='coerce').astype("Int64"))
    .assign(age = lambda x: x['age'].replace(sex_mapping))
    .drop(columns=["TIME_PERIOD"])
)

df_unemployment['new_columns'] = df_unemployment['sex'].str.lower() + "_unemployment"

df_unemployment_pivot =  df_unemployment.pivot_table(
    index=['geo', "year"],                 
    columns='new_columns',          
    values='unemployment_rate',             
    aggfunc='first'                       
)
df_unemployment_pivot.head(10)

new_columns   females_unemployment  males_unemployment  total_unemployment
geo     year                                                              
Austria 2015                   5.3                 6.1                 5.7
        2016                   5.6                 6.5                 6.0
        2017                   5.0                 5.9                 5.5
        2018                   4.7                 5.0                 4.9
        2019                   4.4                 4.6                 4.5
        2020                   5.2                 5.5                 5.4
        2021                   6.1                 6.3                 6.2
        2022                   4.5                 4.9                 4.8
        2023                   4.9                 5.3                 5.1
        2024                   4.7                 5.6                 5.2

In [23]:

df_poverty = data_frames["poverty_risk"]
age_mapping = {
    "Less than 16 years": "LT_16Y",
    "From 16 to 64 years": "16Y-64Y",
    "65 years or over": "GE_65Y"}

df_poverty = (
    df_poverty
    .rename(columns={"OBS_VALUE": "poverty_rate"})
    .assign(year = lambda x: pd.to_numeric(x["TIME_PERIOD"], errors='coerce').astype("Int64"))
    .assign(age = lambda x: x['age'].replace(age_mapping))
    .drop(columns=["TIME_PERIOD"])
)
df_poverty['age'] = df_poverty['age'].str.lower() + "_poverty"
df_poverty_pivot =  df_poverty.pivot_table(
    index=['geo', "year"],                 
    columns='age',          
    values='poverty_rate',             
    aggfunc='first'                       
)
df_poverty_pivot.head(10)


age           16y-64y_poverty  ge_65y_poverty  lt_16y_poverty  total_poverty
geo     year                                                                
Albania 2017             58.5            61.6            65.4           60.3
        2018             53.7            60.7            59.0           55.7
        2019             51.7            56.0            54.5           52.8
        2020             46.1            52.5            52.0           48.1
        2021             46.6            53.8            52.3           48.7
        2022             44.6            51.1            48.9           46.4
        2023             41.9            48.6            47.6           44.0
Austria 2015             16.7            15.9            22.6           17.4
        2016             17.7            16.8            21.8           18.1
        2017             17.3            16.6            24.6           18.3

In [ ]:
GDP_COLUMNS = ["geo", "TIME_PERIOD", "OBS_VALUE"]
df_gdp = data_frames["real_gdp"][GDP_COLUMNS]

df_gdp = (
    df_gdp
    .rename(columns={"OBS_VALUE": "gdp"})
    .assign(year = lambda x: pd.to_numeric(x["TIME_PERIOD"], errors='coerce').astype("Int64"))
    .assign(gdp = lambda x: x["gdp"].astype("Int64"))
    .set_index(["geo", "year"])
    .drop(columns = ["TIME_PERIOD"])
)
df_gdp.head(10)

gdp
geo     year      
Albania 2000  2120
        2001  2330
        2002  2440
        2003  2580
        2004  2730
        2005  2880
        2006  3080
        2007  3300
        2008  3550
        2009  3680

In [ ]:
WEALTH_COLUMNS=["geo","TIME_PERIOD","stk_flow","OBS_VALUE"]
df_wealth = data_frames["wealth_distribution_gini"][WEALTH_COLUMNS]


df_wealth = (
    df_wealth
    .rename(columns={"OBS_VALUE": "gini_wealth", "stk_flow": "type"})
    .assign(year = lambda x: pd.to_numeric(x["TIME_PERIOD"], errors='coerce').astype("Int64"))
    .drop(columns=["TIME_PERIOD"])
)

df_wealth_pivot =  df_wealth.pivot_table(
    index=['geo', "year"],                 
    columns='type',          
    values='gini_wealth',             
    aggfunc='first'                       
)
df_wealth_pivot.head(10)


type           Expenditure  Income  Net wealth  Savings
geo      year                                          
Austria  2010         33.5    35.4        75.0     67.9
         2015         34.3    34.3        72.3     65.7
         2020         33.5    34.0        69.0     64.7
Belgium  2010         32.8    34.8        60.7     72.8
         2015         33.8    34.3        58.5     67.3
         2020         34.0    33.5        61.8     61.1
Bulgaria 2010         31.8    41.2         NaN     57.8
         2015         34.2    43.2         NaN     71.4
         2020         33.7    46.6         NaN     69.3
Croatia  2010         32.9    39.3         NaN     75.6

In [8]:
MANIFESTO_COLUMNS = ['countryname', 'edate', 'partyname', 'pervote',  'party', 'parfam', 'rile', 'planeco', 'welfare', 'markeco', 'per107', 'per202', 'per416_2']
manifesto_filtered = data_frames["manifesto"][MANIFESTO_COLUMNS]

In [ ]:

renaming_map = {
    'partyname': 'party_name',
    'pervote': 'vote_share_percent',
    'countryname': 'country',
    'edate': 'election_date',
    'party': 'party_code',

    'rile': 'right_left_position',
    'planeco': 'planned_economy',
    'welfare': 'welfare_state',
    'markeco': 'market_economy',
    
    'per107': 'inter_positive',
    'per202': 'democracy',    
    'per416_2': 'sustainability_positive', 
    
}

manifesto_pre =manifesto_filtered.rename(columns=renaming_map)
manifesto_pre["election_date"] = pd.to_datetime(
    manifesto_pre["election_date"], 
    format='%d/%m/%Y', 
    errors='coerce' 
)            
manifesto_pre.head(10)


,country,election_date,party_name,vote_share_percent,party_code,parfam,right_left_position,planned_economy,welfare_state,market_economy,inter_positive,democracy,sustainability_positive
0,Sweden,1944-09-17,Communist Party of Sweden,10.300,11220,20,9.60,1.9,0.0,1.900,0.0,0.0,NaN
1,Sweden,1944-09-17,Social Democratic Labour Party,46.500,11320,30,-37.80,3.3,33.4,2.200,4.4,1.1,NaN
2,Sweden,1944-09-17,People’s Party,12.900,11420,40,9.50,3.2,14.3,6.400,3.2,6.4,NaN
3,Sweden,1944-09-17,Right Party,15.800,11620,60,28.00,1.8,10.6,22.800,3.5,0.0,NaN
4,Sweden,1944-09-17,Agrarian Party,13.600,11810,80,23.81,0.0,0.0,19.048,0.0,0.0,NaN
5,Sweden,1948-09-19,Communist Party of Sweden,6.312,11220,20,-44.00,8.0,18.0,2.000,0.0,10.0,NaN
6,Sweden,1948-09-19,Social Democratic Labour Party,46.132,11320,30,-33.40,6.7,20.8,5.800,1.7,5.0,NaN
7,Sweden,1948-09-19,People’s Party,22.749,11420,40,22.90,2.3,7.0,13.700,1.2,3.4,NaN
8,Sweden,1948-09-19,Right Party,12.343,11620,60,56.30,0.0,0.0,50.000,0.0,3.1,NaN
9,Sweden,1948-09-19,Agrarian Party,12.385,11810,80,-4.90,3.5,13.5,9.200,0.0,2.1,NaN


In [28]:
eu_countries = [
    'Austria',
    'Belgium',
    'Bulgaria',
    'Croatia',
    'Cyprus',
    'Czechia',
    'Denmark',
    'Estonia',
    'Finland',
    'France',
    'Germany',
    'Greece',
    'Hungary',
    'Ireland',
    'Italy',
    'Latvia',
    'Lithuania',
    'Luxembourg',
    'Malta',
    'Netherlands',
    'Poland',
    'Portugal',
    'Romania',
    'Slovakia',
    'Slovenia',
    'Spain',
    'Sweden'
]
eu_country_abbreviations = {
    'Austria': 'AT',
    'Belgium': 'BE',
    'Bulgaria': 'BG',
    'Croatia': 'HR',
    'Cyprus': 'CY',
    'Czechia': 'CZ',
    'Denmark': 'DK',
    'Estonia': 'EE',
    'Finland': 'FI',
    'France': 'FR',
    'Germany': 'DE',
    'Greece': 'GR',
    'Hungary': 'HU',
    'Ireland': 'IE',
    'Italy': 'IT',
    'Latvia': 'LV',
    'Lithuania': 'LT',
    'Luxembourg': 'LU',
    'Malta': 'MT',
    'Netherlands': 'NL',
    'Poland': 'PL',
    'Portugal': 'PT',
    'Romania': 'RO',
    'Slovakia': 'SK',
    'Slovenia': 'SI',
    'Spain': 'ES',
    'Sweden': 'SE'
}
manifesto_eu = manifesto_pre.loc[manifesto_pre["country"].isin(eu_countries)].copy()
manifesto_eu["geo"] = manifesto_eu["country"].map(eu_country_abbreviations)
manifesto_eu["election_date"] = pd.to_datetime(manifesto_eu["election_date"])
manifesto_eu["year"] = manifesto_eu["election_date"].dt.year
manifesto_eu.head(10)

,country,election_date,party_name,vote_share_percent,party_code,parfam,right_left_position,planned_economy,welfare_state,market_economy,inter_positive,democracy,sustainability_positive,geo,year
0,Sweden,1944-09-17,Communist Party of Sweden,10.300,11220,20,9.60,1.9,0.0,1.900,0.0,0.0,NaN,SE,1944
1,Sweden,1944-09-17,Social Democratic Labour Party,46.500,11320,30,-37.80,3.3,33.4,2.200,4.4,1.1,NaN,SE,1944
2,Sweden,1944-09-17,People’s Party,12.900,11420,40,9.50,3.2,14.3,6.400,3.2,6.4,NaN,SE,1944
3,Sweden,1944-09-17,Right Party,15.800,11620,60,28.00,1.8,10.6,22.800,3.5,0.0,NaN,SE,1944
4,Sweden,1944-09-17,Agrarian Party,13.600,11810,80,23.81,0.0,0.0,19.048,0.0,0.0,NaN,SE,1944
5,Sweden,1948-09-19,Communist Party of Sweden,6.312,11220,20,-44.00,8.0,18.0,2.000,0.0,10.0,NaN,SE,1948
6,Sweden,1948-09-19,Social Democratic Labour Party,46.132,11320,30,-33.40,6.7,20.8,5.800,1.7,5.0,NaN,SE,1948
7,Sweden,1948-09-19,People’s Party,22.749,11420,40,22.90,2.3,7.0,13.700,1.2,3.4,NaN,SE,1948
8,Sweden,1948-09-19,Right Party,12.343,11620,60,56.30,0.0,0.0,50.000,0.0,3.1,NaN,SE,1948
9,Sweden,1948-09-19,Agrarian Party,12.385,11810,80,-4.90,3.5,13.5,9.200,0.0,2.1,NaN,SE,1948


In [ ]:

manifesto_eu['is_election'] = True

def fill_years(group):
    # Standard setup
    group['year'] = group['year'].astype(int)
    group = group.drop_duplicates(subset=['year'])
    
    current_geo = group['geo'].iloc[0]
    current_party = group['party_name'].iloc[0]
    
    full_years = pd.RangeIndex(group['year'].min(), group['year'].max() + 1, name='year')
    group = group.set_index('year').reindex(full_years)
    
    group['geo'] = current_geo
    group['party_name'] = current_party
    
   
    if 'is_election' in group.columns:
        group['is_election'] = group['is_election'].fillna(False)
    group = group.ffill()

    return group.reset_index()

# Apply
manifesto_eu_filled = manifesto_eu.groupby(['geo', 'party_name'], group_keys=False).apply(fill_years)
manifesto_eu_filled.head(100)

C:\Users\ssick\AppData\Local\Temp\ipykernel_70096\1395605450.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group['is_election'] = group['is_election'].fillna(False)
C:\Users\ssick\AppData\Local\Temp\ipykernel_70096\1395605450.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group['is_election'] = group['is_election'].fillna(False)
C:\Users\ssick\AppData\Local\Temp\ipykernel_70096\1395605450.py:43: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objec

In [37]:
right_dfs = [df_poverty_pivot, df_gdp, df_prices, df_unemployment_pivot, df_wealth_pivot]
manifesto_fin = manifesto_eu.set_index(["geo", "year"])
merged_df = manifesto_fin

# Perform sequential left merges
for right_df in right_dfs:
    
    merged_df = pd.merge(
        left=merged_df,
        right=right_df,
        how='left',
        left_index=True, 
        right_index=True,                  
        suffixes=('', '_new_data')        
    )

In [38]:
merged_df.head(100)

country election_date                      party_name  \
geo year                                                         
SE  1944  Sweden    1944-09-17       Communist Party of Sweden   
    1944  Sweden    1944-09-17  Social Democratic Labour Party   
    1944  Sweden    1944-09-17                  People’s Party   
    1944  Sweden    1944-09-17                     Right Party   
    1944  Sweden    1944-09-17                  Agrarian Party   
...          ...           ...                             ...   
    1998  Sweden    1998-09-21          Liberal People’s Party   
    1998  Sweden    1998-09-21             Christian Democrats   
    1998  Sweden    1998-09-21        Moderate Coalition Party   
    1998  Sweden    1998-09-21                    Centre Party   
    2002  Sweden    2002-09-15             Green Ecology Party   

          vote_share_percent  party_code  parfam  right_left_position  \
geo year                                                                
SE  1944              10.300       11220      20                9.600   
    1944              46.500       11320      30              -37.800   
    1944              12.900       11420      40                9.500   
    1944              15.800       11620      60               28.000   
    1944              13.600       11810      80               23.810   
...                      ...         ...     ...                  ...   
    1998               4.720       11420      40               14.286   
    1998              11.770       11520      50                4.790   
    1998              22.900       11620      60               37.425   
    1998               5.130       11810      80               11.570   
    2002               4.646       11110      10              -26.087   

          planned_economy  welfare_state  market_economy  ...  total_poverty  \
geo year                                                  ...                  
SE  1944            1.900          0.000           1.900  ...            NaN   
    1944            3.300         33.400           2.200  ...            NaN   
    1944            3.200         14.300           6.400  ...            NaN   
    1944            1.800         10.600          22.800  ...            NaN   
    1944            0.000          0.000          19.048  ...            NaN   
...                   ...            ...             ...  ...            ...   
    1998            0.000         10.714          13.095  ...            NaN   
    1998            0.000         26.347          16.168  ...            NaN   
    1998            0.000          4.790          27.844  ...            NaN   
    1998            2.479         14.876           9.091  ...            NaN   
    2002            0.621         22.981           0.000  ...            NaN   

           gdp  consumer_prices  females_unemployment  males_unemployment  \
geo year                                                                    
SE  1944  <NA>              NaN                   NaN                 NaN   
    1944  <NA>              NaN                   NaN                 NaN   
    1944  <NA>              NaN                   NaN                 NaN   
    1944  <NA>              NaN                   NaN                 NaN   
    1944  <NA>              NaN                   NaN                 NaN   
...        ...              ...                   ...                 ...   
    1998  <NA>              NaN                   NaN                 NaN   
    1998  <NA>              NaN                   NaN                 NaN   
    1998  <NA>              NaN                   NaN                 NaN   
    1998  <NA>              NaN                   NaN                 NaN   
    2002  <NA>              NaN                   NaN                 NaN   

          total_unemployment  Expenditure  Income  Net wealth  Savings  
geo year                                                                
SE  1944   